In [32]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///D:/bld/scikit-learn_1652976858669/work


In [33]:
import os
import pickle
import pandas as pd

## Q1. Notebook

In [34]:
year = 2021
month = 2
taxi_type = 'fhv'

In [35]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

C:\Users\Timur.Kamaliev\Anaconda3\envs\mlops-zoomcamp\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Timur.Kamaliev\Anaconda3\envs\mlops-zoomcamp\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [36]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [37]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet')

In [38]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [39]:
print(f'Mean predicted duration: {y_pred.mean():.3f}')

Mean predicted duration: 16.192


## Q2. Preparing the output

In [40]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [41]:
df['prediction'] = y_pred

In [42]:
df_result = df[['ride_id', 'prediction']]

In [43]:
df_result.head(10)

,ride_id,prediction
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206
6,2021/02_6,15.757813
7,2021/02_7,14.105704
9,2021/02_9,12.910885
10,2021/02_10,23.465230
11,2021/02_11,15.105853


In [44]:
os.makedirs(f'data/{taxi_type}', exist_ok=True)

In [45]:
output_file = f'data/{taxi_type}/{year:04d}_{month:02d}_result.parquet'

In [46]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [48]:
!ls data/$taxi_type -la

total 19252
drwxr-xr-x 1 Timur.Kamaliev 1049089        0 РёСЋРЅ 27 00:22 .
drwxr-xr-x 1 Timur.Kamaliev 1049089        0 РёСЋРЅ 27 00:22 ..
-rw-r--r-- 1 Timur.Kamaliev 1049089 19711435 РёСЋРЅ 27 00:22 2021_02_result.parquet


## Q3. Creating the scoring script

In [49]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1650 bytes to starter.py
